In [1]:
import numpy as np
import os
import json

In [2]:
from filtool_script import FiltoolExec

In [3]:
fil_exec = FiltoolExec('/home/rsenzel/Code/inception/dev_injector/fb/controlfb2_uhf.fil', 'mmgps_search.json', 'test_dir', 1)

In [4]:
fil_exec.processing_args

{'rfi_flags': 'kadaneF 8 4 zdot',
 'temp_filesystem': '/beegfs/',
 'fscrunch': 1,
 'snr_threshold': 8.0,
 'nharmonics': '4',
 'fft_length': 4194304,
 'start_accel': -105.0,
 'end_accel': 105.0,
 'accel_tol': 1.11,
 'candidate_limit': 1000,
 'ddplan': ' 0.000 103.800 0.03 1 3460 0.7544\r\n 103.800 167.300 0.05 2 1270 0.1384\r\n 167.300 318.500 0.10 4 1512 0.08242\r\n 318.500 500.100 0.20 8 908 0.02475',
 'overrides': 'both',
 'birdie_list': '1000.00:0.05,1500.00:0.05,1300.00:0.05',
 'channel_mask': '932.00:961.00',
 'ram_limit': 24.0}

In [49]:
import xml.etree.ElementTree as ET
import json

In [54]:
with open('/home/rsenzel/Code/inception/offline_injector_nextflow/xml_files/injection_report_inj_000001.json', 'r') as file:
    pars = json.load(file)

psr = pars['pulsars']

In [111]:
tree1a = ET.parse('/home/rsenzel/Code/inception/offline_injector_nextflow/xml_files/data_000001_inj_000001_overview_dm_0.000000_103.800000.xml')
tree2a = ET.parse('/home/rsenzel/Code/inception/offline_injector_nextflow/xml_files/overview_dm_0.000000_103.800000.xml')

tree1b = ET.parse('/home/rsenzel/Code/inception/offline_injector_nextflow/xml_files/data_000001_inj_000001_overview_dm_103.800000_167.300000.xml')
tree2b = ET.parse('/home/rsenzel/Code/inception/offline_injector_nextflow/xml_files/overview_dm_103.800000_167.300000.xml')

tree1c = ET.parse('/home/rsenzel/Code/inception/offline_injector_nextflow/xml_files/data_000001_inj_000001_overview_dm_167.300000_318.500000.xml')
tree2c = ET.parse('/home/rsenzel/Code/inception/offline_injector_nextflow/xml_files/overview_dm_167.300000_318.500000.xml')

tree1d = ET.parse('/home/rsenzel/Code/inception/offline_injector_nextflow/xml_files/data_000001_inj_000001_overview_dm_318.500000_500.100000.xml')
tree2d = ET.parse('/home/rsenzel/Code/inception/offline_injector_nextflow/xml_files/overview_dm_318.500000_500.100000.xml')

root1a = tree1a.getroot()
root2a = tree2a.getroot()
root1b = tree1b.getroot()
root2b = tree2b.getroot()
root1c = tree1c.getroot()
root2c = tree2c.getroot()
root1d = tree1d.getroot()
root2d = tree2d.getroot()

In [113]:
def xml_to_dict(element):
    # Convert element attributes and text to a dictionary
    data = {element.tag: {} if element.attrib else None}
    children = list(element)
    
    # Process each child
    if children:
        dd = {}
        for dc in map(xml_to_dict, children):
            for k, v in dc.items():
                if k in dd:
                    if not isinstance(dd[k], list):
                        dd[k] = [dd[k]]
                    dd[k].append(v)
                else:
                    dd[k] = v
        data = {element.tag: dd}
    if element.attrib:
        data[element.tag].update(('@' + k, v) for k, v in element.attrib.items())
    if element.text:
        text = element.text.strip()
        if children or element.attrib:
            if text:
                data[element.tag]['#text'] = text
        else:
            data[element.tag] = text
    return data

In [117]:
cands1a = xml_to_dict(root1a)['peasoup_search']['candidates']['candidate']
cands2a = xml_to_dict(root2a)['peasoup_search']['candidates']['candidate']
cands1b = xml_to_dict(root1b)['peasoup_search']['candidates']['candidate']
cands2b = xml_to_dict(root2b)['peasoup_search']['candidates']['candidate']
cands1c = xml_to_dict(root1c)['peasoup_search']['candidates']['candidate']
cands2c = xml_to_dict(root2c)['peasoup_search']['candidates']['candidate']
cands1d = xml_to_dict(root1d)['peasoup_search']['candidates']['candidate']
cands2d = xml_to_dict(root2d)['peasoup_search']['candidates']['candidate']

cands_inj = [*cands1a, *cands1b, *cands1c, *cands1d]
cands_ = [*cands2a, *cands2b, *cands2c, *cands2d]

In [118]:
inj_p0 = [psr_i['PX'][0] for psr_i in psr]
inj_dm = [psr_i['DM'] for psr_i in psr]
inj_snr = [psr_i['SNR'] for psr_i in psr]
inj_p0, inj_dm, inj_snr

([2.49736279882014,
  0.005029998441627896,
  0.029441233314046255,
  0.023319415821309163,
  0.01746540112908594,
  0.015199383305185218,
  0.015557874177102932,
  0.019476140097705392,
  0.0013852768121955213,
  0.0140967591560671],
 [109.07311425431257,
  73.78790443468304,
  490.4624307052723,
  385.96933901889776,
  286.04736409991517,
  247.36878400229944,
  253.4878523333087,
  320.3685982194541,
  11.576276621958035,
  228.54813042252462],
 [14.727958901288893,
  12.446894832141126,
  39.38342986377518,
  32.62832090627218,
  26.168718487267242,
  23.668285026411276,
  24.06386116094117,
  28.387464935399052,
  8.425133034146782,
  22.451596310143007])

In [116]:
cands1a

[{'period': '0.00449386581419261',
  'opt_period': '0',
  'dm': '99.870002746582',
  'acc': '82.9064636230469',
  'nh': '0',
  'snr': '10.0899257659912',
  'folded_snr': '0',
  'is_adjacent': '1',
  'is_physical': '1',
  'ddm_count_ratio': '1',
  'ddm_snr_ratio': '1',
  'nassoc': '7',
  'byte_offset': '0',
  '@id': '0'},
 {'period': '2.49834498130547',
  'opt_period': '0',
  'dm': '103.769996643066',
  'acc': '78.0695724487305',
  'nh': '2',
  'snr': '9.97921371459961',
  'folded_snr': '0',
  'is_adjacent': '1',
  'is_physical': '1',
  'ddm_count_ratio': '1',
  'ddm_snr_ratio': '1',
  'nassoc': '24955',
  'byte_offset': '196',
  '@id': '1'},
 {'period': '0.00403834853629584',
  'opt_period': '0',
  'dm': '98.129997253418',
  'acc': '14.6851167678833',
  'nh': '1',
  'snr': '9.97436809539795',
  'folded_snr': '0',
  'is_adjacent': '1',
  'is_physical': '1',
  'ddm_count_ratio': '1',
  'ddm_snr_ratio': '1',
  'nassoc': '1',
  'byte_offset': '599144',
  '@id': '2'},
 {'period': '0.0010951

In [110]:
cands1[1], cands2[1]

({'period': '2.49834498130547',
  'opt_period': '0',
  'dm': '103.769996643066',
  'acc': '78.0695724487305',
  'nh': '2',
  'snr': '9.97921371459961',
  'folded_snr': '0',
  'is_adjacent': '1',
  'is_physical': '1',
  'ddm_count_ratio': '1',
  'ddm_snr_ratio': '1',
  'nassoc': '24955',
  'byte_offset': '196',
  '@id': '1'},
 {'period': '0.00109510986390266',
  'opt_period': '0',
  'dm': '47.6100006103516',
  'acc': '-13.3175163269043',
  'nh': '0',
  'snr': '9.99514484405518',
  'folded_snr': '0',
  'is_adjacent': '0',
  'is_physical': '1',
  'ddm_count_ratio': '0.666666686534882',
  'ddm_snr_ratio': '0.696503043174744',
  'nassoc': '2',
  'byte_offset': '244',
  '@id': '1'})

In [13]:
ddplan = list(fil_exec.create_DDplan())
filplan = fil_exec.create_filplan(ddplan)

In [14]:
ddplan = fil_exec.create_DDplan()

In [26]:
np.unique(np.random.randint(0, 11, size=10000))

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [30]:
arr = np.array(['abs', 'rty', 'fer', 'fre'])

with open("output.txt", "w") as file:
    # Add a newline character to each string and write to file
    file.writelines(line + "\n" for line in arr)

In [31]:
from scipy.stats import truncnorm

In [34]:
trvs = truncnorm(a=10, b=20, loc=25, scale=5).rvs

In [42]:
trvs(random_state=0)

np.float64(75.39255995729471)

In [43]:
2**23

8388608

In [11]:
f'overview_dm_0.000000_103.800000.xml'

'overview_dm_0.000000_103.800000.xml'

In [12]:
xml_file_names = [f'overview_dm_{dm_range.low_dm:.6f}_{dm_range.high_dm:.6f}.xml' for dm_range in ddplan]
xml_file_names

['overview_dm_0.000000_103.800000.xml',
 'overview_dm_103.800000_167.300000.xml',
 'overview_dm_167.300000_318.500000.xml',
 'overview_dm_318.500000_500.100000.xml']

In [6]:
for i in ddplan:
    print(i)

DMRange(low_dm=0.0, high_dm=517.8, dm_step=0.3, tscrunch=1.0)
DMRange(low_dm=517.8, high_dm=834.8, dm_step=0.5, tscrunch=2.0)
DMRange(low_dm=834.8, high_dm=1589.8, dm_step=1.0, tscrunch=4.0)
DMRange(low_dm=1589.8, high_dm=3001.8, dm_step=2.0, tscrunch=8.0)


In [45]:
dm_range = np.linspace(ddplan[0].low_dm, ddplan[0].high_dm, int((ddplan[0].high_dm-ddplan[0].low_dm)/ddplan[0].dm_step), endpoint=False)
dm1 = np.linspace(ddplan[1].low_dm, ddplan[1].high_dm, int((ddplan[1].high_dm-ddplan[1].low_dm)/ddplan[1].dm_step), endpoint=False)

In [82]:
dm_trials_list = []
tscrunch_list = []
for i, dm_range in enumerate(ddplan):
    n_trial = int((dm_range.high_dm-dm_range.low_dm)/dm_range.dm_step)
    n_trial, endpoint = (n_trial + 1, True) if i == len(ddplan)-1 else (n_trial, False)
    dm_list_i = np.linspace(dm_range.low_dm, dm_range.high_dm, n_trial, endpoint=endpoint)
    tscrunch = np.ones_like(dm_list_i) * dm_range.tscrunch
    dm_trials_list.append(dm_list_i)
    tscrunch_list.append(tscrunch)

In [83]:
dm_trials = np.concat(dm_trials_list)
tscrunch = np.concat(tscrunch_list)
tscrunch

array([1., 1., 1., ..., 8., 8., 8.])

In [66]:
def find_n_nearest(dm_arr, tscrunch_arr, target, n):
    nearest_indices = np.argsort(np.abs(dm_arr - target))[:n]
    dm_values = dm_arr[nearest_indices]
    tscrunch_values = tscrunch_arr[nearest_indices]
    return dm_values, tscrunch_values

In [71]:
a, b = find_n_nearest(dm_trials, tscrunch, 517.7, 5)
a, b

(array([517.8, 517.5, 517.2, 518.3, 516.9]), array([2., 1., 1., 2., 1.]))

In [81]:
np.where(np.array([dm_range.tscrunch for dm_range in ddplan] )== 2)[0][0]

np.int64(1)

In [73]:
a[np.where(b == 1)]

array([517.5, 517.2, 516.9])

In [50]:
my_dms = np.random.uniform(50, 1500, size=10)
nearest = 5
dm_search = np.zeros((len(my_dms), nearest))
my_dms, dm_search

(array([1072.13331736, 1374.67901752, 1121.25312234, 1487.44617915,
        1061.48080809, 1298.42985136, 1143.93419058,  585.98544057,
         226.25915701,  530.1386847 ]),
 array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]))